## Notebook 11.1: *denovo* Genome assembly 


### Learning objectives: 

By the end of this notebook you will:

+ Be familiar with assembled genome file formats. 
+ Know where to find assembled genomes and their raw files online.
+ Be able to compute genome size from kmer counts of Illumina short reads. 
+ Experience running Illumina, and Illumina+PacBio hybrid *de novo* assemblies.  

### Assigned readings: 

**Read these papers carefully, we will discuss in class:**

+ Li, Fay-Wei, and Alex Harkess. “A Guide to Sequence Your Favorite Plant Genomes.” Applications in Plant Sciences 6, no. 3 (n.d.): e1030. https://doi.org/10.1002/aps3.1030.



+ Sumpter, Nicholas, Margi Butler, and Russell Poulter. “Single-Phase PacBio De Novo Assembly of the Genome of the Chytrid Fungus Batrachochytrium Dendrobatidis, a Pathogen of Amphibia.” Microbiology Resource Announcements 7, no. 21 (November 2018). https://doi.org/10.1128/MRA.01348-18.



**Please skim the paper below in terms of the abstract, figures, and tables.**
+ Giordano, Francesca, Louise Aigrain, Michael A. Quail, Paul Coupland, James K. Bonfield, Robert M. Davies, German Tischler, et al. “De Novo Yeast Genome Assemblies from MinION, PacBio and MiSeq Platforms.” Scientific Reports 7, no. 1 (June 21, 2017): 3935. https://doi.org/10.1038/s41598-017-03996-z.




### Software used in this notebook
We will use three Python packages and also a number of command-line programs that have been installed using the conda package manager. The commands to install the software are shown below for reference, but are commented out. You do not need to run the installation commands since the software is already installed for you. 

In [1]:
import toyplot
import numpy as np
import pandas as pd

In [2]:
# conda install numpy pandas
# conda install eaton-lab::toyplot   
# conda install bioconda::cutadapt      # read adapter/quality trimmer
# conda install bioconda::jellyfish     # kmer counting
# conda install bioconda::spades        # debruijn graph assembler
# conda install bioconda::minimap2      # short/long read mapper/aligner
# conda install bioconda::miniasm       # OLC-based de novo assembler
# conda install bioconda::racon         # polisher: consensus calls on long read assembly   

### Computational resources for genome assembly

Genome assembly is very computationally intensive. Specifically, *de novo* genome assembly -- the process of assembling a genome without any *a priori* guide. This stands in contrast to reference-guided assembly methods which are used to assemble a genome by mapping reads to an existing close relative's genome. The *de novo* process is incredibly more demanding both in terms of the amount of input data and computational power that is required. 

In particular, *de novo* assembly requires enormous amounts of RAM (i.e., accessible memory). This type of resource is not easily shared across nodes of a computing cluster (e.g., a University HPC system), and so instead clusters typically contain a few special nodes, designated *high-memory* nodes, that are  best for this process. As an example, even small bacterial genomes will typically require 8GB or more of RAM, and human genomes require 100-500Gb, depending on the software being used. Much larger genomes obviously require even more. High memory nodes often have 1Tb of RAM or more. 

In this notebook and the next we will complete several genome assemblies using real data. However, to avoid the need for significant computational resources most of the computationally intensive code is commented out, meaning you will not actually run it. We've made all of the files available to you so that you can see the results that are produced, and look at the files to interpret their format and meaning. 

For these exercises we will work with a relatively small genome and a smallish number of sequenced reads with the goal of assembling the Eukaryotic genome *Batrachochytrium dendrobatidis*, a Chytrid fungus that is an ecologically important pathogen of amphibians, and the subject of one of your assigned readings. 

## The *Bd* genome
We will assemble the *Bd* chytrid fungus genome from the assigned Sumpter et al. (2018) paper. I selected this paper because it provides a very concise report of a genome assembly and also where to find the associated data.

The first draft genome assembly of *Bd* genome was sequenced by the US DOE Joint Genome Institute using Sanger sequencing to 8.74X coverage in 2011. That assembly is in 127 scaffolds (N50 scaffold size of 1.48Mb) and 510 contigs (N50 contig size 318K) (https://www.ncbi.nlm.nih.gov/assembly/GCF_000203795.1). In your assigned p aper by Sumpter et al. a new draft genome is assembled using PacBio data alone (and some additional scaffolding methods). Our exercise in this notebook will be to re-assemble this data set using their data, and to explore alternative assembly methods. 


<div class="alert alert-success">
    <b>[1] Question:</b> 
    What are the assembly statistics for the initial draft genome assembled by Sumpter et al. using PacBio data (before they do additional scaffolding)? Answer in markdown below.  
</div>

In [3]:
#answer here:   


## Download sequence data (.fastq) files

The paper by Sumpter et al. includes a section called *Data availability* describing the accession number under which the data have been archived. This is typical for any genome paper. The genome assembly is deposited under the accession ID `QUAD00000000`, with the particular *version* of the genome assembled in this paper given the ID QUAD01000000. Future assemblies of the same data, perhaps using better software, could be uploaded and associated with this project, but given new accession IDs. This accession identifier can be used to query the assembly and retrieve the contigs which are labeled QUAD01000001-QUAD01000063. Follow the link in the paper to the genome project page [https://www.ncbi.nlm.nih.gov/nuccore/QUAD00000000](https://www.ncbi.nlm.nih.gov/nuccore/QUAD00000000). There you can find additional links to the Project accession, or the BioSample accession, all of which can tell you more about the project and how to access the data. As described in the Data Availability section of the paper, there were 10 other strains of Bd also sequenced in this study (using Illumina short read data) which are also linked under the same Project ID (PRJNA483086), but are not linked under the same Genome Assembly (QUAD00000000). By following links on the project page we can find pages for the sequenced individuals (SRXs) and sequencing runs (SRRs). I know, the accession IDs are hard to keep straight. These last IDs link to the actual sequence data files. For example, this page has details about the PacBio data we will be assembling: [https://www.ncbi.nlm.nih.gov/sra/SRX4676189[accn]](https://www.ncbi.nlm.nih.gov/sra/SRX4676189[accn]). 

Large fastq sequence data files are stored online in a weird format called `.sra` which includes a lot of connected metadata. This can be a pain when you just want to access the fastq files. We have installed a suite of command-line tools called the `sra-tools`, that allow you to query these databases using accession IDs, download the sequence data, and convert it into the proper format all in one command. From these tools the only one we really need is called  `fastq-dump`. The commands below will download the PacBio data for Bd, and Illumina data for one of the other strains included in the study which was shown to be identical to the one sequenced with PacBio. The PacBio data is all single reads, whereas the Illumina data is paired-end reads (separate R1 and R2 files).

*This download would take about 20 minutes*, so instead we downloaded the data files ahead of time for you. Therefore the code is commented out. 

In [4]:
%%bash

## Download Bd RTP6 PacBio reads
#fastq-dump SRR7825134

## Download Bd RTP5 Illumina HiSeq 2000 paired-end reads
#fastq-dump SRR7825135 --split-files


### The fastq data files
You have seen fastq data files before, and the PacBio data is no different from Illumina fastq data files, except the reads are just longer. How good of an assembly of the *Bd* genome can we get from short read data alone? How does this compare to long-read only or hybrid assembly approaches?

<div class="alert alert-success">
    <b>[2] Question:</b> 
    How many sequenced reads are in each of the raw fastq data files (SRR7825134.fasta, SRR7825135_1.fasta and SRR7825135_2.fasta)? Use any method (bash or Python) to calculate and print the answer in the cell below. 
</div>

In [57]:
# your answer here


### PacBio read lengths
Let's use some Python code to look at the read length distribution in the PacBio data. We can see that this data is much better than the PacBio reads we looked at a few sessions ago. The average read length is around 7500bp in length (7.5kb) and many reads longer than 20kb!

In [86]:
# get lengths of each read
with open("./SRR7825134.fastq") as indata:
    lengths = np.array([len(next(indata)) for line in indata if line[0] == "@"])

In [100]:
# plot as a barplot
toyplot.bars(
    np.histogram(lengths, bins=20),
    width=350, 
    height=300,
    xlabel="read length", 
    ylabel="number of reads"
);

# print statistics
print("Read length statistics")
print("mean: {:.0f}bp".format(np.mean(lengths)))
print("median: {:.0f}bp".format(np.median(lengths)))

Read length statistics
mean: 7517bp
median: 6207bp


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc0635ee9881849c7b355afa846cf666d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 10000 20000 30000 40000 50000 read length 0 10000 20000 30000 40000 number of reads

### Trim reads for quality and adapters

Filtering and trimming is an important step for any genome assembly. The cleaner your reads are the better and faster the genome will assemble. The command below uses the program `cutadapt` to trim Illumina adapters from the reads and trim from the 3' end when quality falls below a set threshold based on the quality scores in the fastq data files. To keep our notebook easy to read we will suppress the outputs printed to screen by `cutadapt`, which includes stats about how many reads were trimmed or removed, and write those stats to a report file instead. You can tell from the size difference between the trimmed and untrimmed files though that it a fair amount of the data was cleaned up. The trimming step here takes only a few minutes, but again, we've commented it out for effiency. 

We will not clean up the PacBio data files for now. These are expected to have lower quality scores, and so instead of trimming them to throw away the bad parts it is more common to try to correct the errors either as a first step before assembly, or during assembly. This can involve mapping short-read data to the long reads. An example program for this is the tool `proovread`, or it can involve analyzing the long-reads by themselves, their overlaps, quality scores, and composition, as is done in the software `canu`. 

Let's start by trimming the short read data. 


In [90]:
# cutadapt parameters explained:
# -q: trim 3' when quality < 10
# -a: trim adapters from R1
# -A: trim adapters from R2
# -o: output R1 filename
# -p: output R2 filename

In [24]:
%%bash

# cutadapt \
#     -q 10 \
#     -a AGATCGGAAGAGC -A AGATCGGAAGAGC \
#     -o SRR7825135_1.trim.fastq \
#     -p SRR7825135_2.trim.fastq \
#     SRR7825135_1.fastq \
#     SRR7825135_2.fastq > trim-report.txt

The report produced by `cutadapt` includes statistics about how many reads were trimmed based on low quality or adapter contamination. Open the report at [this link](https://raw.githubusercontent.com/genomics-course/11-assembly/master/notebooks/trim-report.txt) to see the results. In the command above we named the new trimmed read files `SRR7825135_1.trim.fastq` and `SRR7825135_2.trim.fastq`. These are the files we will use going forward. 

<div class="alert alert-success">
    <b>[3] Question:</b> 
    From the statistics in the cutadapt report (see link above), what percentage of read1 and read2s were trimmed for adapters? How many based in total were trimmed based on quality scores?  Answer in markdown below. 
</div>

In [16]:
# your answer here


### Genome size and complexity
Your assigned reading by Li and Harkess titled "A guide to sequence your favorite plant genomes" describes some particular difficulties of assembling plant genomes, but more broadly the advise from this paper can be generalized to many other organisms with difficult to assemble genomes as well. They make the important point that before starting any genome sequencing project it is important to learn as much as you can about your organism, as this should inform the type of approach that you take, and your expectations about the result you will get. 

One of the first things you can do when you get some initial shotgun sequenced Illumina data is to investigate some of these same questions: How big is the genome, how variable, and what ploidy (has it experienced genome duplications)? It turns out we can learn answers to these questions by examining the distribution of kmer counts in the raw read data itself, without even having to map, align, or assemble it. 

<div class="alert alert-success">
    <b>[4] Question:</b> 
    In Figure 2 in Li and Harkess what characteristics of the kmer distribution plots are informative about heterozygosity and repetitiveness? Why does one distribution have one hump while the other has two? Answer in Markdown below.     
</div>

In [20]:
# answer here:

### Jellyfish: kmer counting tool
The [jellyfish](https://github.com/gmarcais/Jellyfish/tree/master/doc) program is a tool used to efficiently count kmers from a genome fasta file or fastq sequenced read files. It works much faster than the Python code we wrote last week to find all kmers in a sequence and is quite easy to use. It turns out we can learn a lot from examining the distribution of kmer counts. The quote below is from the Jellyfish paper by the authors, Marçais and Kingford (2011): 
    
<blockquote>

Given a string S, we are often interested in counting the number of occurrences in S of every substring of length k. These length-k substrings are called k-mers and the problem of determining the number of their occurrences is called k-mer counting.

Counting the k-mers in a DNA sequence is an important step in many applications. For example, genome assemblers using the overlap-layout-consensus paradigm, such as the Celera (Miller et al., 2008; Myers et al., 2000) and Arachne (Jaffe et al., 2003) assemblers, use k-mers shared by reads as seeds to find overlaps. Statistics on the number of occurrences of each k-mer are first computed and used to filter out which k-mers are used as seeds. Such k-mer count statistics are also used to estimate the genome size: if a large fraction of k-mers occur c times, we can estimate the sequencing coverage to be approximately c and derive an estimate of the genome size from c and the total length of the reads. In addition, in most short-read assembly projects, errors are corrected in the sequencing reads to improve the quality of the final assembly. For example, Kelley et al. (2010) use k-mer frequencies to assess the likelihood that a misalignment between reads is a sequencing error or a genuine difference in sequence. A third application is the detection of repeated sequences, such as transposons, which play an important biological role. De novo repeat annotation techniques find candidate regions based on k-mer frequencies (Campagna et al., 2005; Healy et al., 2003; Kurtz et al., 2008; Lefebvre et al., 2003). The counts of k-mers are also used to seed fast multiple sequence alignment (Edgar, 2004). Finally, k-mer distributions can produce new biological insights directly. Sindi et al. (2008) used k-mers frequencies with large k (20 ≤ k ≤ 100) to study the mechanisms of sequence duplication in genomes.
</blockquote>

Running jellyfish on the raw data files produces a `.jf` formatted file with information about the kmers in the data. Here we run jellyfish on the full size data files because kmer counting requires ~30X or more to show good peaks. This would take a few minutes to run. Again, we've commented it out so you can proceed to the output files. 

In [26]:
# jellyfish parameters explained:
# -m kmer size
# -s genome size estimate
# -t number of threads to use
# -C switch to count 'canonical kmers' (reverse complement)
# -o output location
# <(...) this decompresses the file as it is passed to jellyfish

In [36]:
%%bash

# jellyfish count \
#     -m 21 -s 40M -t 10 -C \
#     -o 21mer.jf \
#     SRR7825135_1.fastq \
#     SRR7825135_2.fastq


After running jellyfish we can then extract a histogram from the .jf file using the `histo` function, which returns the histogram as a text table showing the number of unique kmers that occurred N times. You can see in the bash script above we direct the histogram to a file named `21mer.hist.csv`. Again we put this file online [here](https://raw.githubusercontent.com/genomics-course/11-assembly/master/notebooks/21mer.hist.csv) so that you do not have to calculate it yourself. We will load in the file below to plot the kmer distribution. 

In [38]:
%%bash

# jellyfish histo 21mer.jf > 21mer.hist.csv

Let's now use Python to load the kmer count data with the Pandas library and plot it with the Toyplot library. We can see clearly that there is a single best peak at around 55X coverage or so. We can assume that this peak represents a single copy of the genome -- all possible unique kmers in the genome have been sequenced to 55X coverage on average -- and that any other peaks represent heterozygous sites in the genome (thus the kmers at those regions only come up half as often), errors, repetitions, and other such noise. The density of our single-copy peak stretches from about 30 to 80X. 

In [40]:
# load the histogram data as a table
url = "https://raw.githubusercontent.com/genomics-course/11-assembly/master/notebooks/21mer.hist.csv"
df = pd.read_csv(url, sep=" ", names=["bins", "counts"])

# plot the histogram
toyplot.fill(
    df.counts[2:200], 
    width=500, 
    height=300,
    ylabel="number of unique kmers", 
    xlabel="kmer coverage depth",
);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tf6e76c37d00341f3bbd34c8c7b60b1f1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 300.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="50.0,250.0 52.0,250.0 54.0,250.0 56.0,250.0 58.0,250.0 60.0,250.0 62.0,250.0 64.0,250.0 66.0,250.0 68.0,250.0 70.0,250.0 72.0,250.0 74.0,250.0 76.0,250.0 78.0,250.0 80.0,250.0 82.0,250.0 84.0,250.0 86.0,250.0 88.0,250.0 90.0,250.0 92.0,250.0 94.0,250.0 96.0,250.0 98.0,250.0 100.0,250.0 102.0,250.0 104.0,250.0 106.0,250.0 108.0,250.0 110.0,250.0 112.0,250.0 114.0,250.0 116.0,250.0 118.0,250.0 120.0,250.0 122.0,250.0 124.0,250.0 126.0,250.0 128.0,250.0 130.0,250.0 132.0,250.0 134.0,250.0 136.0,250.0 138.0,250.0 140.0,250.0 142.0,250.0 144.0,250.0 146.0,250.0 148.0,250.0 150.0,250.0 152.0,250.0 154.0,250.0 156.0,250.0 158.0,250.0 160.0,250.0 162.0,250.0 164.0,250.0 166.0,250.0 168.0,250.0 170.0,250.0 172.0,250.0 174.0,250.0 176.0,250.0 178.0,250.0 180.0,250.0 182.0,250.0 184.0,250.0 186.0,250.0 188.0,250.0 190.0,250.0 192.0,250.0 194.0,250.0 196.0,250.0 198.0,250.0 200.0,250.0 202.0,250.0 204.0,250.0 206.0,250.0 208.0,250.0 210.0,250.0 212.0,250.0 214.0,250.0 216.0,250.0 218.0,250.0 220.0,250.0 222.0,250.0 224.0,250.0 226.0,250.0 228.0,250.0 230.0,250.0 232.0,250.0 234.0,250.0 236.0,250.0 238.0,250.0 240.0,250.0 242.0,250.0 244.0,250.0 246.0,250.0 248.0,250.0 250.0,250.0 252.0,250.0 254.0,250.0 256.0,250.0 258.0,250.0 260.0,250.0 262.0,250.0 264.0,250.0 266.0,250.0 268.0,250.0 270.0,250.0 272.0,250.0 274.0,250.0 276.0,250.0 278.0,250.0 280.0,250.0 282.0,250.0 284.0,250.0 286.0,250.0 288.0,250.0 290.0,250.0 292.0,250.0 294.0,250.0 296.0,250.0 298.0,250.0 300.0,250.0 302.0,250.0 304.0,250.0 306.0,250.0 308.0,250.0 310.0,250.0 312.0,250.0 314.0,250.0 316.0,250.0 318.0,250.0 320.0,250.0 322.0,250.0 324.0,250.0 326.0,250.0 328.0,250.0 330.0,250.0 332.0,250.0 334.0,250.0 336.0,250.0 338.0,250.0 340.0,250.0 342.0,250.0 344.0,250.0 346.0,250.0 348.0,250.0 350.0,250.0 352.0,250.0 354.0,250.0 356.0,250.0 358.0,250.0 360.0,250.0 362.0,250.0 364.0,250.0 366.0,250.0 368.0,250.0 370.0,250.0 372.0,250.0 374.0,250.0 376.0,250.0 378.0,250.0 380.0,250.0 382.0,250.0 384.0,250.0 386.0,250.0 388.0,250.0 390.0,250.0 392.0,250.0 394.0,250.0 396.0,250.0 398.0,250.0 400.0,250.0 402.0,250.0 404.0,250.0 406.0,250.0 408.0,250.0 410.0,250.0 412.0,250.0 414.0,250.0 416.0,250.0 418.0,250.0 420.0,250.0 422.0,250.0 424.0,250.0 426.0,250.0 428.0,250.0 430.0,250.0 432.0,250.0 434.0,250.0 436.0,250.0 438.0,250.0 440.0,250.0 442.0,250.0 444.0,250.0 444.0,249.21386666666666 442.0,249.21386666666666 440.0,249.1829333333333 438.0,249.17226666666667 436.0,249.10986666666665 434.0,249.0941333333333 432.0,249.09333333333333 430.0,249.03786666666667 428.0,248.99786666666665 426.0,248.96266666666665 424.0,248.96266666666665 422.0,248.94213333333332 420.0,248.89333333333335 418.0,248.86880000000002 416.0,248.82026666666667 414.0,248.79973333333336 412.0,248.77413333333334 410.0,248.7165333333333 408.0,248.72666666666666 406.0,248.67946666666666 404.0,248.6288 402.0,248.66479999999999 400.0,248.61306666666667 398.0,248.58800000000002 396.0,248.53146666666666 394.0,248.57706666666667 392.0,248.57279999999997 390.0,248.61306666666667 388.0,248.55173333333335 386.0,248.4936 384.0,248.55253333333332 382.0,248.53573333333333 380.0,248.51386666666664 378.0,248.52559999999997 376.0,248.528 374.0,248.54959999999997 372.0,248.57146666666668 370.0,248.59893333333335 368.0,248.60266666666666 366.0,248.5829333333333 364.0,248.57706666666667 362.0,248.57173333333333 360.0,248.55226666666667 358.0,248.5608 356.0,248.4976 354.0,248.50159

### Genome size estimation
We can estimate the genome size by taking the total number of kmers (multiple the coverage times the number of unique kmers in each bin) and dividing this by the mean coverage (peak position). We get an estimate of ~30Mb, which is close to the current estimated value of the *Bd* genome of 24Mb.  

In [45]:
# calculate genome size estimate
int(np.sum(df.counts[1:] * df.bins[1:]) / 55)

30657414

### *de novo* assembly methods

There are many programs available today for genome assembly. They vary in terms of requirements of the input data, their speed, RAM requirements, and the resulting accuracy of the assemblies. For some types of data, or sizes of genomes, some tools will work better than others. Because *de novo* assembly is such a time consuming process it is difficult to compare many tools and so people often just use a program that others suggest. It's worth reading a few bioinformatics articles that compare assembly tools to learn more about which one might be best for your project. Your assigned *skim* reading this week by Giordano et al. is one such paper. It compares several assembly tools on a variety of data sets. The tool we will use below, [spades](http://cab.spbu.ru/software/spades/), has been popular for years, and continually updated to keep pace with advancing sequencing technologies. It is a *de Bruijn* graph based assembly method that is best for short-read assembly, but which recently added support for *hybrid* assemblies that combine short-read contig assembly with scaffolding by using long-reads (HybridSpades). We will try these approaches here. 

### Spades 
Any genome assembly software tool will include documentation about how to use it, and sometimes additional tutorials. Let's take a look at the Spades manual. In particular, take a look at the section [here, titled Spades' performance](http://spades.bioinf.spbau.ru/release3.11.1/manual.html#sec1.3). We can see when spades is applied to a test dataset of an E. coli genome (a very small genome) it takes about 8m to run, uses 8.1 Gb of RAM, and uses 1.5 Gb of disk space. This is when the input data is composed of 28M reads that are 2x100bp (paired). Our data set is a similar size in terms of numbers of reads, and so likely requires similar resources. 

Again, because we don't have that many resources available to use the code below is commented out and we will focus on examining the output files. 

### *de novo* assembly with spades: Illumina only

The command below runs a *de novo* assembly using Illumina data only. It first runs a pre-processing read correction step, then the main assembly of the data, and finally a post-processing contig gap filling correction step. While it is running it prints a report of its progress to the screen and to a log file. This reports which step it is on, how long it has been running, and how much RAM it is consuming. Look at the log file [here](...) for this assembly.

In [50]:
# spades parameters explained:

# -k: if not set, then kmer size is automatically selected
# -o: output directory name
# -1: read1 inputs
# -2: read2 inputs
# -m: memory limit in Gb
# -t: number of CPU threads to use

# --only-error-correction: if set, performs read error correction only. 
# --only-assembler: if set, then performs assembly only without corrections.
# --careful: if set, turns on post-processing mismatch-correction (gap filling)
# if none of these three are set then only error-correction and assembly are done by default.

In [92]:
%%bash

# spades.py \
#     -o assembly_spades/ \
#     -1 SRR7825135_1.trim.fastq \
#     -2 SRR7825135_2.trim.fastq \
#     -m 64 \
#     -t 20


<div class="alert alert-success">
    <b>[5] Question:</b> 
    From looking at the log file (linked to above) for this assembly run with spades, answer the following questions in Markdown below. 
    How many threads and how much memory was made available to the program (see the top of the log file)? 
    How long did the read correction step take to run? How many bases were corrected in total? What kmer sizes were used in the assembly? How long did the assembly step take to run?
</div>

In [47]:
# answer here

### Success, you just assembled a genome
Now let's try another method, by including some additional PacBio data to span the gaps between contigs in the assembly to see if we get longer scaffolds. 


### *de novo* HYBRID assembly with spades: Illumina & PacBio
To run spades in hybrid mode we can use the same command as above with just the addition of the `--pacbio` flag (or --nanopore if we had nanopore data) and the input fastq file for the long read data. The log file for this assembly can be found [here](...). 

In [91]:
%%bash

# spades.py \
#     -o assembly_spades_hybrid/ \
#     -1 SRR7825135_1.fastq \
#     -2 SRR7825135_2.fastq \
#     --pacbio SRR7825134.fastq \
#     -m 64 \
#     -t 20


<div class="alert alert-success">
    <b>[6] Question:</b> 
    From looking at the log file (linked to above) for this assembly run with HybridSpades, answer the following questions in Markdown below. How long did the entire assembly take to complete? How much time approximately was spent on the step "Long read assembly"?
    
</div>

In [52]:
# answer here

### Get assembly quality statistics using QUAST
The [quast](http://quast.bioinf.spbau.ru/manual.html) program is used to calculate genome quality statistics and compare assemblies. It calculates N50 contig and scaffold sizes (something we've done before), and reports BUSCO scores (number of single-copy genes present) which we've read about before. For this we'll provide a GFF file telling it where genes are located in the genome, and then it will 

In [56]:
%%bash

# # download the busco gene set to this computer
# quast-download-busco

# # run quast to compare assemblies and find BUSCO conserved genes in both
# quast.py \
#     assembly_spades/scaffolds.fasta \
#     assembly_spades_hybrid/scaffolds.fasta \
#     -o quast_results \
#     --conserved-genes-finding \ 
#     --fungus
    

### After running quast
After running quast you can open a summary of the results in HTML format from the output directory. I've provided a link to it for you that should open in a new browser tab. 

+ [de novo Illumina assembly statistics](./results_assembly_1/latest/report.html)
+ [de novo hybrid Illumina/PacBio assembly statistics](./results_assembly_2/latest/report.html)

<div class="alert alert-success">
    <b>[5] Question:</b> 
    Which assembly do you think is better? Which statistics do you base this on?
</div>

### Busco scores
The BUSCO scores examine how many single-copy orthologs that are commonly conserved across all fungi are found in our assembled genomes. The Quast analysis reports this for each assembly separately. Results files were uploaded and can be seen here:

+ [de novo Illumina BUSCO statistics](...)
+ [de novo hybrid Illumina/PacBio assembly BUSCO statistics](...)

<div class="alert alert-success">
    <b>[6] Question:</b> 
    Name three possible next steps you could take to improve the genome assembly?
</div>

### Question from last time: What are PacBio 'subreads'

Found answer here: http://seqanswers.com/forums/showthread.php?t=34790


Subreads are the individual sequence reads that are determined in real time from a template on the sequencer. These would correspond to the stretch of DNA between the adapter sequence.

CCS reads are the result of doing a consensus base calling from subreads that are all from the same template. If the template was short enough, then the polymerase will loop entirely from the beginning back to the beginning, sequencing the adapter before it starts on the template again. In this case, the same template piece of DNA is sequenced more than once, which means that the sequence data generated from each pass can be used to determine one single consensus sequence with higher base quality than the raw subreads. Thus, the quality distribution that you see is correct between the subreads and the CCS reads

